In [ ]:
import pandas as pd
import numpy as np
import shap
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.ensemble import ExtraTreesRegressor, ExtraTreesClassifier, GradientBoostingRegressor, GradientBoostingClassifier
from utils import bayescv, reg_scores, cls_scores, plot_permutation_importance

In [ ]:
pd.set_option('display.max_columns',1000)
pd.options.display.float_format = '{:.3f}'.format
sns.set_theme(style='whitegrid')
model_types = ['extratree', 'gradientboost']
thres = 1.03
n_iter = 30
random_state = 42

cls = model_types[0]
zscore = False
bayes = True

## Load BF2 data

In [ ]:
load_bf2_df = pd.read_csv('csv/BF2_R.csv')
load_bf2_df['APOE'].replace({22.: '22', 23.: '23', 24.: '24', 33.: '33', 34.: '34', 44.: '44'}, inplace=True)
if zscore:
    load_bf1_df = pd.read_csv('csv/BF1_R_Z.csv')
    load_bf1_df['APOE'].replace({22.: '22', 23.: '23', 24.: '24', 33.: '33', 34.: '34', 44.: '44'}, inplace=True)

In [ ]:
ptau217 = ['Plasma WashU %P-tau217',
           'Plasma Lilly P-tau217',
           'CSF Lilly P-tau217',
           'CSF WashU P-tau217']

common = ['CSF Aβ42/Aβ40',
          'Age',
          'APOE',
          'ADAS',
          'Education',
          'Sex',
          'Cognitive status',
          'MMSE',
          'CSF Abnormal Ratio',
          'Diagnosis status',
          'fnc_ber_com_composite']

cd_drop = [ 'CSF Aβ42/Aβ40',
                'Age',
                'APOE',
                'ADAS',
                'Education',
                'Sex',
                'Cognitive status',
                'MMSE',
                'CSF Abnormal Ratio',
                'Diagnosis status']

name = ['BF2-P-MS','BF2-P-IA','BF2-C-IA','BF2-C-MS']

In [ ]:
ptau217_index = 0
features = [ptau217[ptau217_index]] + common
select_df = load_bf2_df[features]
select_df = select_df.dropna(how='any').reset_index(drop=True)

In [ ]:
labels = []
for i in range(select_df.shape[0]):
    if select_df['fnc_ber_com_composite'][i] <= thres:
        labels.append('0')
    else:
        labels.append('1')
select_df['labels'] = labels
select_df.info()

In [ ]:
neg_idx = select_df['fnc_ber_com_composite']<thres
pos_idx = (1-neg_idx).astype('bool')
neg_df = select_df[neg_idx]
pos_df = select_df[pos_idx]

## Classification

In [ ]:
neg_tv_df, neg_test_df = train_test_split(neg_df, test_size=0.2, random_state=random_state)
pos_tv_df, pos_test_df = train_test_split(pos_df, test_size=0.2, random_state=random_state)
tv_df = pd.concat([neg_tv_df, pos_tv_df])
test_df = pd.concat([neg_test_df, pos_test_df])

tv_df = tv_df.drop(cd_drop,axis=1)
test_df = test_df.drop(cd_drop,axis=1)

X_train = tv_df.drop(['labels','fnc_ber_com_composite'], axis=1)
y_train = tv_df['labels']
X_test = test_df.drop(['labels','fnc_ber_com_composite'], axis=1)
y_test = test_df['labels']

In [ ]:
X_train.info()

In [ ]:
if cls == 'extratree':
    model = ExtraTreesClassifier()
elif cls == 'gradientboost':
    model = GradientBoostingClassifier()

In [ ]:
if bayes:
    opt = bayescv(X_train, y_train, n_iter, model, random_state, cls=cls)
    best_param = dict(opt.best_params_)
else:
    best_param = {'max_depth': 4,
 'min_samples_leaf': 2,
 'min_samples_split': 4,
 'n_estimators': 500}
best_param

In [ ]:
if cls == 'extratree':
    cls_model = ExtraTreesClassifier(**best_param, random_state=random_state)
elif cls == 'gradientboost':
    cls_model = GradientBoostingClassifier(**best_param, random_state=random_state)
cls_model.fit(X_train,y_train)

In [ ]:
result_df = pd.DataFrame(cls_scores(cls_model, X_train, y_train, X_test, y_test), index=['Train_Acc', 'Test_Acc', 'Train_MacroF1', 'Test_MacroF1']).T
result_df

In [ ]:
RocCurveDisplay.from_estimator(cls_model, X_test, y_test)
plt.title('ROC of Classification between Amyloid Negative and Positive')
plt.show()

In [ ]:
sns.set_theme(style='white')
cm = confusion_matrix(y_test, cls_model.predict(X_test), labels=cls_model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=cls_model.classes_)
disp.plot()
plt.show()

In [ ]:
ConfusionMatrixDisplay.from_estimator(cls_model, X_train, y_train)

In [ ]:
fea_imp = pd.DataFrame(columns=['AVG_Importance'], index=[i for i in X_train.columns])
fea_imp['AVG_Importance'] = cls_model.feature_importances_
fea_imp = fea_imp.sort_values(by="AVG_Importance" , inplace=False, ascending=True) 

fig = plt.figure(figsize=(8,6))
ax = fea_imp.iloc[:,:].plot(kind='barh', color=['b'],figsize=(10,6))
# bar_datalabel(ax)
ax.set_xlabel('Weight')
ax.set_xlim(0, np.max(fea_imp['AVG_Importance'].values)*1.1) # expand xlim to make labels easier to read
plt.title('Feature Importance of the Classifier')
plt.show()

In [ ]:
cls_fea_df = pd.DataFrame(fea_imp).rename(columns={'AVG_Importance':'Classification'})
cls_fea_df

In [ ]:
mdi_importances = pd.Series(cls_model.feature_importances_, index=X_train.columns)
tree_importance_sorted_idx = np.argsort(cls_model.feature_importances_)
tree_indices = np.arange(0, len(cls_model.feature_importances_)) + 0.5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
mdi_importances.sort_values().plot.barh(ax=ax1)
ax1.set_xlabel("Gini importance")
plot_permutation_importance(cls_model, X_train, y_train, ax2, random_state)
ax2.set_xlabel("Decrease in accuracy score")
fig.suptitle(
    "Impurity-based vs. permutation importances on multicollinear features (train set)"
)
_ = fig.tight_layout()

### SHAP

In [ ]:
shap_df = select_df.drop(cd_drop,axis=1).reset_index(drop=True)
X_shap = shap_df.drop(['labels','fnc_ber_com_composite'], axis=1)
y_shap = shap_df['labels']
y_pred_shap = cls_model.predict(X_shap)

In [ ]:
shap.initjs()
explainer = shap.TreeExplainer(cls_model)
shap_values = explainer.shap_values(X_shap)
shap_exp = explainer(X_shap)

In [ ]:
plt.title('SHAP Values on the Whole Dataset')
shap.summary_plot(shap_values, X_shap, max_display=20, show=True, cmap='plasma', plot_size=[12,6], class_names=y_shap.unique())
# savefig_name = crop[8:-7] + 'RF_SHAP_impact.png'
# plt.savefig(savefig_name,format='png')
plt.show()

## Negative

In [ ]:
if cls == 'extratree':
    model = ExtraTreesRegressor()
elif cls == 'gradientboost':
    model = GradientBoostingRegressor()

In [ ]:
neg_idx = select_df['fnc_ber_com_composite']<=thres
pos_idx = (1-neg_idx).astype('bool')
neg_df = select_df[neg_idx]
pos_df = select_df[pos_idx]

tv_df, test_df = train_test_split(neg_df, test_size=0.2, random_state=random_state)

tv_df = tv_df.drop(cd_drop,axis=1)
test_df = test_df.drop(cd_drop,axis=1)

X_train = tv_df.drop(['labels', 'fnc_ber_com_composite'], axis=1)
y_train = tv_df['fnc_ber_com_composite']

X_test = test_df.drop(['labels', 'fnc_ber_com_composite'], axis=1)
y_test = test_df['fnc_ber_com_composite']

In [ ]:
X_train.info()

In [ ]:
if 1:
    opt = bayescv(X_train, y_train, n_iter, model, random_state, cls=cls)
    best_param = dict(opt.best_params_)
else:
    best_param = {
    'max_depth': 3,
    'min_samples_leaf': 2,
    'min_samples_split': 2,
    'n_estimators': 60}
best_param

In [ ]:
if cls == 'extratree':
    neg_model = ExtraTreesRegressor(**best_param, random_state=random_state)
elif cls == 'gradientboost':
    neg_model = GradientBoostingRegressor(**best_param, random_state=random_state)
neg_model.fit(X_train,y_train)

In [ ]:
result_df = pd.DataFrame(reg_scores(neg_model, X_train, y_train, X_test, y_test), index=['Train_R2', 'Test_R2', 'Train_MAPE', 'Test_MAPE']).T
result_df

In [ ]:
## Observation vs. Prediction
sns.set_theme(style='whitegrid', palette=sns.color_palette('deep'))
y_pred = neg_model.predict(X_test)
fig, ax = plt.subplots(1,1,figsize=(7,5))
max_value = max(y_test.max(), y_pred.max(), y_train.max(), neg_model.predict(X_train).max())
min_value = min(y_test.min(), y_pred.min(), y_train.min(), neg_model.predict(X_train).min())

sns.scatterplot(x=y_train, y=neg_model.predict(X_train),  ax=ax, color=sns.color_palette('deep')[0])
sns.scatterplot(x=y_test, y=y_pred,  ax=ax, marker='s', color=sns.color_palette('deep')[3])
sns.regplot(x=y_train, y=neg_model.predict(X_train),  ax=ax, line_kws={'color':'darkblue'}, color=sns.color_palette('deep')[0],scatter=False)
sns.regplot(x=y_test, y=y_pred, ax=ax, line_kws={'color':'darkred'}, color=sns.color_palette('deep')[3],scatter=False)
l = min(y_train.min(), neg_model.predict(X_train).min()),max(y_train.max(), neg_model.predict(X_train).max())
ax.plot(l,l,'--',c='black', linewidth=0.8)
ax.set_title('Observation vs. Prediction within the Amyloid Negative Group')
ax.set_xlabel('Observation [SUVR]')
ax.set_ylabel('Prediction [SUVR]')
ax.annotate("Train R2={:.3f}".format(result_df['Train_R2'][0]), (0.98, 0.815))
ax.annotate("Test R2={:.3f}".format(result_df['Test_R2'][0]), (0.98, 0.8))
ax.legend(['Train', 'Test'], loc='upper right')

plt.show()

In [ ]:
sns.set_theme(style='whitegrid', palette=sns.color_palette("tab10")[0:])
fea_imp = pd.DataFrame(columns=['AVG_Importance'], index=[i for i in X_train.columns])
fea_imp['AVG_Importance'] = neg_model.feature_importances_
fea_imp = fea_imp.sort_values(by="AVG_Importance" , inplace=False, ascending=True) 

row_names = {'PL_pT217T217percentmean_WashU_2023':'Plasma %p-tau217',
             'CSF_Ab42_Ab40_ratio_imputed_Elecsys_2020_2022':'CSF AB42/AB40',
             'age':'Age',
             'apoe_genotype_baseline_variable':'APOE'}
fea_imp = fea_imp.rename(index = row_names)

fig = plt.figure(figsize=(8,6))
ax = fea_imp.iloc[:,:].plot(kind='barh',figsize=(10,6))
# bar_datalabel(ax)
ax.set_xlabel('Weight')
ax.set_xlim(0, np.max(fea_imp['AVG_Importance'].values)*1.1) # expand xlim to make labels easier to read
plt.title('Feature Importance within the Amyloid Negative Group')
plt.show()

## Positive

In [ ]:
tv_df, test_df = train_test_split(pos_df, test_size=0.2, random_state=random_state)

if zscore:
    zs_feature = ptau217[ptau217_index]
    control_df = tv_df[((tv_df['Cognitive status'] == 'Normal') | 
                        (tv_df['Cognitive status'] == 'SCD')) & (tv_df['CSF Abnormal Ratio'] == 0)]
    z_mean = control_df[zs_feature].mean()
    z_std = control_df[zs_feature].std()
    tv_df[zs_feature] = (tv_df[zs_feature]-z_mean)/z_std
    test_df[zs_feature] = (test_df[zs_feature]-z_mean)/z_std
    if ptau217_index in [0,1]:
        bf1_df = load_bf1_df.drop('CSF Lilly p-tau217',axis=1)
        bf1_df.rename(columns = {'Plasma Lilly p-tau217': select_df.columns[0]}, inplace = True)
    elif ptau217_index in [2,3,4]:
        bf1_df = load_bf1_df.drop('Plasma Lilly p-tau217',axis=1)
        bf1_df.rename(columns = {'CSF Lilly p-tau217': select_df.columns[0]}, inplace = True)        
    bf1_df = bf1_df.dropna(how='any')
    bf1_df = bf1_df.drop(cd_drop,axis=1)
    X_bf1 = bf1_df.drop(['fnc_ber_com_composite'], axis=1)
    y_bf1 = bf1_df['fnc_ber_com_composite']

tv_df = tv_df.drop(cd_drop,axis=1)
test_df = test_df.drop(cd_drop,axis=1)

X_train = tv_df.drop(['labels', 'fnc_ber_com_composite'], axis=1)
y_train = tv_df['fnc_ber_com_composite']

X_test = test_df.drop(['labels', 'fnc_ber_com_composite'], axis=1)
y_test = test_df['fnc_ber_com_composite']

In [ ]:
n_iter = 30
cls = 'extratree'
if cls == 'extratree':
    model = ExtraTreesRegressor()
elif cls == 'gradientboost':
    model = GradientBoostingRegressor()

In [ ]:
if bayes:
    opt = bayescv(X_train, y_train, n_iter, model, random_state=42, cls=cls)
    best_param = dict(opt.best_params_)
else:
    best_param = {'max_depth': 6,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 72}
best_param

In [ ]:
if cls == 'extratree':
    pos_model = ExtraTreesRegressor(**best_param, random_state=random_state)
elif cls == 'gradientboost':
    pos_model = GradientBoostingRegressor(**best_param, random_state=random_state)
pos_model.fit(X_train,y_train)

In [ ]:
## rs=42, plasma
result_df = pd.DataFrame(reg_scores(pos_model, X_train, y_train, X_test, y_test), index=['Train_R2', 'Test_R2', 'Train_MAPE', 'Test_MAPE']).T
result_df

In [ ]:
## Observation vs. Prediction
if zscore:
    sns.set_theme(style='whitegrid', palette=sns.color_palette('deep'))
    y_pred = pos_model.predict(X_test)
    fig, ax = plt.subplots(1,1,figsize=(7,5))
    max_value = max(y_test.max(), y_pred.max(), y_train.max(), pos_model.predict(X_train).max())
    min_value = min(y_test.min(), y_pred.min(), y_train.min(), pos_model.predict(X_train).min())

    sns.scatterplot(x=y_train, y=pos_model.predict(X_train),  ax=ax, color=sns.color_palette('deep')[0])
    sns.scatterplot(x=y_test, y=y_pred,  ax=ax, marker='s', color=sns.color_palette('deep')[3])
    sns.scatterplot(x=y_bf1, y=pos_model.predict(X_bf1),  ax=ax, marker='*', color=sns.color_palette('deep')[2])
    sns.regplot(x=y_train, y=pos_model.predict(X_train),  ax=ax, line_kws={'color':'darkblue'}, color=sns.color_palette('deep')[0], scatter=False)
    sns.regplot(x=y_test, y=y_pred, ax=ax, line_kws={'color':'darkred'}, color=sns.color_palette('deep')[3], scatter=False)
    sns.regplot(x=y_bf1, y=pos_model.predict(X_bf1), ax=ax, line_kws={'color':'darkgreen'}, color=sns.color_palette('deep')[3], scatter=False)
    l = min(y_train.min(), pos_model.predict(X_train).min()),max(y_train.max(), pos_model.predict(X_train).max())
    ax.plot(l,l,'--',c='black', linewidth=0.8)
    ax.set_xlabel('Observation')
    ax.set_ylabel('Prediction')
    ax.annotate("Train R2={:.3f}".format(result_df['Train_R2'][0]), (2.0, 1.18))
    ax.annotate("Test R2={:.3f}".format(result_df['Test_R2'][0]), (2.0, 1.1))
    ax.annotate("BF1 R2={:.3f}".format(pos_model.score(X_bf1, y_bf1)), (2.0, 1.02))
    ax.legend(['Train', 'Test', 'BF1'], loc='upper right')
    ax.set_title('Observation vs. Prediction within the Amyloid Positive Group')
    plt.show()

In [ ]:
## Observation vs. Prediction
if not zscore:
    sns.set_theme(style='whitegrid', palette=sns.color_palette('deep'))
    y_pred = pos_model.predict(X_test)
    fig, ax = plt.subplots(1,1,figsize=(7,5))
    max_value = max(y_test.max(), y_pred.max(), y_train.max(), pos_model.predict(X_train).max())
    min_value = min(y_test.min(), y_pred.min(), y_train.min(), pos_model.predict(X_train).min())

    sns.scatterplot(x=y_train, y=pos_model.predict(X_train),  ax=ax, color=sns.color_palette('deep')[0])
    sns.scatterplot(x=y_test, y=y_pred,  ax=ax, marker='s', color=sns.color_palette('deep')[3])
    sns.regplot(x=y_train, y=pos_model.predict(X_train),  ax=ax, line_kws={'color':'darkblue'}, color=sns.color_palette('deep')[0], scatter=False)
    sns.regplot(x=y_test, y=y_pred, ax=ax, line_kws={'color':'darkred'}, color=sns.color_palette('deep')[3], scatter=False)
    l = min(y_train.min(), pos_model.predict(X_train).min()),max(y_train.max(), pos_model.predict(X_train).max())
    ax.plot(l,l,'--',c='black', linewidth=0.8)
    ax.set_xlabel('Observation [SUVR]')
    ax.set_ylabel('Prediction [SUVR]')
    ax.annotate("Train R2={:.3f}".format(result_df['Train_R2'][0]), (2.0, 1.1))
    ax.annotate("Test R2={:.3f}".format(result_df['Test_R2'][0]), (2.0, 1.02))
    ax.legend(['Train', 'Test'], loc='upper right')
    ax.set_title('Observation vs. Prediction within the Amyloid Positive Group')
    plt.show()

In [ ]:
sns.set_theme(style='whitegrid', palette=sns.color_palette("tab10")[3:])
fea_imp = pd.DataFrame(columns=['AVG_Importance'], index=[i for i in X_train.columns])
fea_imp['AVG_Importance'] = pos_model.feature_importances_
fea_imp = fea_imp.sort_values(by="AVG_Importance" , inplace=False, ascending=True) 

row_names = {'PL_pT217T217percentmean_WashU_2023':'Plasma %p-tau217',
             'CSF_Ab42_Ab40_ratio_imputed_Elecsys_2020_2022':'CSF AB42/AB40',
             'age':'Age',
             'apoe_genotype_baseline_variable':'APOE'}
fea_imp = fea_imp.rename(index = row_names)

fig = plt.figure(figsize=(8,6))
ax = fea_imp.iloc[:,:].plot(kind='barh',figsize=(10,6))
# bar_datalabel(ax)
ax.set_xlabel('Weight')
ax.set_xlim(0, np.max(fea_imp['AVG_Importance'].values)*1.1) # expand xlim to make labels easier to read
plt.title('Feature Importance within the Amyloid Positive Group')
plt.show()

In [ ]:
pos_fea_df = pd.DataFrame(fea_imp).rename(columns={'AVG_Importance':'Regression within Aβ-positive'})
pos_fea_df

In [ ]:
combined = pd.concat([cls_fea_df.T, pos_fea_df.T])
combined

In [ ]:
sns.set_theme(style='whitegrid', palette=sns.color_palette("tab10")[3:])
fig, ax = plt.subplots(1,1,figsize=(7,5))
combined.plot.barh(rot=0, stacked=False, ax=ax, width=.3)
plt.legend(['Plasma %P-tau217', 'CSF Aβ42/Aβ40'], loc='upper right')
plt.xlabel('Weight')
plt.show()

### SHAP

In [ ]:
shap_df = pos_df.drop(cd_drop,axis=1).sort_values(by = 'fnc_ber_com_composite').reset_index(drop=True)
X_shap = shap_df.drop(['labels', 'fnc_ber_com_composite'], axis=1)
y_shap = shap_df['fnc_ber_com_composite']
y_pred_shap = pos_model.predict(X_shap)

In [ ]:
shap.initjs()
explainer = shap.TreeExplainer(pos_model)
shap_values = explainer.shap_values(X_shap)
plt.title('SHAP_values on the Test Set')
shap.summary_plot(shap_values, X_shap, max_display=10, show=True)
plt.gcf().set_size_inches(10,8)
# savefig_name = crop[8:-7] + 'RF_SHAP_impact.png'
# plt.savefig(savefig_name,format='png')

In [ ]:
shap_exp = explainer(X_shap)
output_df = pd.DataFrame()
for idx in range(len(X_shap)):
    details = pd.DataFrame({
        'row_id':idx,
        'feature': X_shap.columns,
        'feature_value': X_shap.iloc[idx,:].values,
        'base_value': shap_exp[idx].base_values,
        'shap_values': shap_exp[idx].values,
        'prediction': y_pred_shap[idx],
        'observation': y_shap[idx],
    })
    output_df = pd.concat([output_df, details])

impact = []
for i in range(len(shap_df)):
    v = np.abs(output_df[output_df['row_id'] == i]['shap_values'])
    imp = list(v/np.sum(v))
    impact = impact + imp
    
output_df['shap_impacts'] = impact

shap_impacts = []
shap_values_plot = []
for chosen_feature in range(len(X_shap.columns)):
    shap_impacts.append(output_df[output_df['feature']==X_shap.columns[chosen_feature]].reset_index(drop=True)['shap_impacts'])
    shap_values_plot.append(output_df[output_df['feature']==X_shap.columns[chosen_feature]].reset_index(drop=True)['shap_values'])

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,7))
i = 0
for impacts in shap_impacts:
    sns.scatterplot(x=y_shap, y=impacts)
    sns.regplot(x=y_shap, y=impacts, order=1, scatter=False, ci=95, ax=ax, label=X_shap.columns[i])
    i += 1
plt.title('Feature Contribution at Different Amyloid PET SUVR')
ax.legend(loc='upper right')
ax.set_xlabel('Amyloid PET SUVR')
ax.set_ylabel('Contribution')

plt.show()